In [1]:
import sys
 
# setting path
sys.path.append('../')

dir = '../data/full data/'

import json
import os
import pandas as pd
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

from utils import elem2sent

import warnings
warnings.filterwarnings('ignore')

In [4]:
meta_data_df = pd.read_excel('../data/category_configuration_marked.xlsx', sheet_name = 'article_names_matching')
title2category = dict(zip(meta_data_df["Article Title"], meta_data_df["Category 2"]))

#read json file
label_address = '../generated_data/category2label.json'
with open(label_address, 'r') as f:
    category2label = json.load(f)


In [5]:
category2label

{'Definition': 0,
 'Promotion and Admission': 1,
 'Most-Favoured-Nation Treatment': 2,
 'Compensation for Losses': 3,
 'Expropriation': 4,
 'Transfers': 5,
 'General Exception Clause': 6,
 'General ISDS Clause': 7,
 'State-to-state Arbitration': 8,
 'Subrogation': 9,
 'Scope of Application': 10,
 'Final provisions': 11,
 'National Treatment': 12,
 'Existing investment': 13,
 'Non-derogation': 14,
 'General Protection and Treatment Clause': 15,
 'Applicable law': 16,
 'Consultation': 17,
 'Taxation': 18,
 'Political Subdivisions': 19,
 'Umbrella Clause': 20,
 'Regional Economic Integration': 21,
 'Amendments': 22,
 'Public Policy Exception': 23,
 'Annex': 24,
 'Relations between States': 25,
 'Entry and Sojourn': 26,
 'Transparency': 27,
 'Public Policy': 28,
 'Compliance with domestic laws': 29,
 'others': 30,
 'Fair and Equitable Treatment': 31,
 'Treaty Language': 32,
 'Permits': 33,
 'Exclusion from ISDS': 34,
 'Non-Conforming Measures': 35,
 'Objectives': 36,
 'Court Access': 37,
 

In [37]:
cnt = 0
df = pd.DataFrame(columns=['treaty_name','article_id', 'predict_category', 'true_category', ])

for file_name in os.listdir(dir):
    try:
        root = ET.parse(dir + file_name).getroot()
        meta_data = root[0]
        doc_body = root[1][2]
    except Exception as e:
        continue

    all_articles_with_title = [_ for _ in doc_body.findall(".//div[@type='article']") ]

    article_id = 0
    for article in all_articles_with_title:
        
        input = ''.join(elem2sent(article))
        sentence_list = [_ for _ in article.itertext() if not _.isspace() and len(_.split()) > 5]
        sentences = "".join(sentence_list)
        category = "NA"
        if 'title' in article.attrib:
            title = article.attrib['title'].lower()
            category = title2category.get(title, 'NA')

        #predict label with model
        prediction_label = 0
        
        pred_category = [k for k, v in category2label.items() if v == prediction_label][0]

        df = df.append({'treaty_name': file_name, 'article_id':article_id, 'predict_category': pred_category,
                         'true_category': category, 'content' : sentences}, ignore_index=True)
        cnt += 1
        article_id += 1

   



In [38]:
df['true_category'] = df['true_category'].apply(lambda x: 'others' if len(df[df['true_category'] == x]) < 5 else x)

In [39]:
df

,treaty_name,article_id,predict_category,true_category,content
0,t1959-1-germany-pakistan-bit-1959.xml,0,Definition,NA,(1) Each contracting State hereafter called in...
1,t1959-1-germany-pakistan-bit-1959.xml,1,Definition,NA,Neither Party shall subject to discriminatory ...
2,t1959-1-germany-pakistan-bit-1959.xml,2,Definition,NA,(1) Investments by nationals or companies of e...
3,t1959-1-germany-pakistan-bit-1959.xml,3,Definition,NA,Either Party shall in respect of all investmen...
4,t1959-1-germany-pakistan-bit-1959.xml,4,Definition,NA,If a claim arising out of a guarantee given fo...
5,t1959-1-germany-pakistan-bit-1959.xml,5,Definition,NA,(1) Transfers under paragraphs (2) or (3) of A...
6,t1959-1-germany-pakistan-bit-1959.xml,6,Definition,NA,If the legislation of either Party or internat...
7,t1959-1-germany-pakistan-bit-1959.xml,7,Definition,NA,"(a) The term ""investment"" shall comprise capit..."
8,t1959-1-germany-pakistan-bit-1959.xml,8,Definition,NA,The present Treaty shall also apply to approve...
9,t1959-1-germany-pakistan-bit-1959.xml,9,Definition,NA,Each Party shall co-operate with the other in ...
